In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'pixel2style2pixel'

In [ ]:
!git clone https://github.com/eladrich/pixel2style2pixel.git $CODE_DIR

In [ ]:
 !wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
 !sudo unzip ninja-linux.zip -d /usr/local/bin/
 !sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

In [ ]:
os.chdir(f'./{CODE_DIR}')

In [ ]:
from argparse import Namespace
import time
import os
import sys
import pprint
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision import utils
from tqdm import tqdm

sys.path.append(".")
sys.path.append("..")

from datasets import augmentations
from utils.common import tensor2im, log_input_image

%load_ext autoreload
%autoreload 2

In [ ]:
from models.stylegan2.model import Generator
generator = Generator(1024,512,8).cuda()
use_550000 = False
if use_550000:
  state_dict = torch.load('/content/drive/MyDrive/StyleGan/550000.pt')
else:
  state_dict = torch.load('/content/drive/MyDrive/StyleGan/stylegan2-ffhq-config-f.pt')
generator.load_state_dict(state_dict['g_ema'], strict=False)

In [ ]:
generator = generator.eval()
NUMBER_OF_IMAGES = 10000

In [ ]:
counter = 0
cur_dir = 0
num_of_images_in_single_loop = 10
for i in tqdm(range(NUMBER_OF_IMAGES//num_of_images_in_single_loop)):
  torch.cuda.empty_cache()
  with torch.no_grad():
    sample_z = torch.randn(num_of_images_in_single_loop, 512, device='cuda')
    sample, latents = generator(
        [sample_z], input_is_latent=False, return_latents=True, truncation= 1, truncation_latent = None
    )

  latents = latents.cpu().detach().numpy()


  for index in range(len(sample)):
    if (counter % 1000) == 0:
      Path(f"/content/drive/My Drive/StyleGan/fake/w/{int(counter / 1000)}").mkdir(parents=True, exist_ok=True)
      Path(f"/content/drive/My Drive/StyleGan/fake/image/{int(counter / 1000)}").mkdir(parents=True, exist_ok=True)
      cur_dir = int(counter / 1000)

    with open(f'/content/drive/My Drive/StyleGan/fake/w/{cur_dir}/{counter}.npy', 'wb') as f:
      np.save(f, latents[index][0])

    utils.save_image(
                sample[index],
                f'/content/drive/My Drive/StyleGan/fake/image/{cur_dir}/{counter}.png',
                nrow=1,
                normalize=True,
                range=(-1, 1),
            )


    counter+=1

In [ ]:
fake_w = np.load(r'/content/drive/MyDrive/StyleGan/fake/w/0/0.npy')
fake_w = fake_w.unsqueeze(0)
fake_w = torch.from_numpy(fake_w)
fake_w = fake_w.float()
images,b = generator([fake_w.cuda()], input_is_latent=False)

In [ ]:
image = images[0].cpu().detach().numpy().transpose(1,2,0)
import matplotlib.pyplot as plt
plt.imshow(image)
plt.show()